In [1]:
import os
import pandas as pd
from tqdm import tqdm
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms
from pathlib import Path

In [2]:
ROOT_DIR   = 'C:/Users/paula/Documents/Repositorio/MURA_prueba/' # directorio raíz
study_type = 'XR_WRIST' # tipo de estudio

DATA_CAT    = ['train', 'valid'] # división de datos
study_label = {'positive': 1, 'negative': 0}

In [3]:
# study_data es un diccionario con los datos de entrenamiento
# y de validación obtenidos de la base de datos

study_data = {}

for phase in DATA_CAT:
    BASE_DIR = ROOT_DIR + '%s/%s/' % (phase, study_type)
    print(BASE_DIR, '\n')
    
    #patients = list(os.walk(BASE_DIR))[0][1] # list of patient folder names
    patients = os.listdir(BASE_DIR)
    study_data[phase] = pd.DataFrame(columns=['Path', 'Count', 'Label'])
    
    i = 0
    for patient in tqdm(patients): # recorremos carpetas de pacientes
        for study in os.listdir(BASE_DIR + patient): # recorremos estudios en cada carpeta de paciente
            label = study_label[study.split('_')[1]] # etiqueta 0 ó 1
            path = BASE_DIR + patient + '/' + study + '/' # ruta del estudio
            study_data[phase].loc[i] = [path, len(os.listdir(path)), label] # añadimos fila
            i+=1

C:/Users/paula/Documents/Repositorio/MURA_prueba/train/XR_WRIST/ 



100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 665.90it/s]


C:/Users/paula/Documents/Repositorio/MURA_prueba/valid/XR_WRIST/ 



100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 586.12it/s]


In [4]:
study_data['train'].head()

,Path,Count,Label
0,C:/Users/paula/Documents/Repositorio/MURA_prue...,3,1
1,C:/Users/paula/Documents/Repositorio/MURA_prue...,4,0
2,C:/Users/paula/Documents/Repositorio/MURA_prue...,3,1
3,C:/Users/paula/Documents/Repositorio/MURA_prue...,3,1


In [5]:
study_data['valid'].head()

,Path,Count,Label
0,C:/Users/paula/Documents/Repositorio/MURA_prue...,4,1
1,C:/Users/paula/Documents/Repositorio/MURA_prue...,2,1
2,C:/Users/paula/Documents/Repositorio/MURA_prue...,3,1
3,C:/Users/paula/Documents/Repositorio/MURA_prue...,3,1
4,C:/Users/paula/Documents/Repositorio/MURA_prue...,1,1


In [6]:
###########################################################
######## CREAMOS DATA LOADER Y CLASE DATASET ASOCIADA
###########################################################

In [7]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import copy
from PIL import Image

In [8]:
print (label)
print (path)

1
C:/Users/paula/Documents/Repositorio/MURA_prueba/valid/XR_WRIST/patient11187/study1_positive/


In [9]:
transform = {
    'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ]),
    'valid': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }


In [10]:
class ImageDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths  # Asegúrate de que file_paths sea una lista
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.file_paths[index])  # Convertir a str antes de abrir la imagen
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.file_paths)

In [11]:
train_file_paths = study_data['train']['Path'].tolist()
train_labels = study_data['train']['Label'].tolist()

valid_file_paths = study_data['valid']['Path'].tolist()
valid_labels = study_data['valid']['Label'].tolist()

train_dataset = ImageDataset(file_paths=train_file_paths, labels=train_labels, transform=transform['train'])
valid_dataset = ImageDataset(file_paths=valid_file_paths, labels=valid_labels, transform=transform['valid'])

#train_dataset = ImageDataset(file_paths =image_path, labels=label, transform=transform['train'])
#valid_dataset = ImageDataset(file_paths =image_path, labels=label, transform=transform['valid'])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=0)


In [12]:
train_num_batches = len(train_dataloader)
valid_num_batches = len(valid_dataloader)

In [13]:
dataset_sizes = {x: len(study_data[x]) for x in DATA_CAT}

In [14]:
print(dataset_sizes)

{'train': 4, 'valid': 5}


In [15]:
from torch.autograd import Variable

In [16]:
def get_count(df, cat):
    '''
    Returns number of images in a study type dataframe which are of abnormal or normal
    Args:
    df -- dataframe
    cat -- category, "positive" for abnormal and "negative" for normal
    '''
    return df[df['Path'].str.contains(cat)]['Count'].sum()

In [17]:
# tai = total abnormal images, tni = total normal images
tai = {x: get_count(study_data[x], 'positive') for x in DATA_CAT}
tni = {x: get_count(study_data[x], 'negative') for x in DATA_CAT}

In [18]:
print('tai:', tai)
print('tni:', tni, '\n')

tai: {'train': 9, 'valid': 13}
tni: {'train': 4, 'valid': 0} 



In [19]:
import torch
from torch.autograd import Variable

In [20]:
def n_p(x):
    '''convert numpy float to Variable tensor float'''    
    return Variable(torch.FloatTensor([x]), requires_grad=False)

In [21]:
Wt1 = {x: n_p(tni[x] / (tni[x] + tai[x])) for x in DATA_CAT}
Wt0 = {x: n_p(tai[x] / (tni[x] + tai[x])) for x in DATA_CAT}

In [22]:
print('Wt0 train:', Wt0['train'])
print('Wt0 valid:', Wt0['valid'])
print('Wt1 train:', Wt1['train'])
print('Wt1 valid:', Wt1['valid'])

Wt0 train: tensor([0.6923])
Wt0 valid: tensor([1.])
Wt1 train: tensor([0.3077])
Wt1 valid: tensor([0.])


In [23]:
############################################################################################
############### ENTRENAMIENTO DEL MODELO
#############################################################################################

In [24]:
# Definición de pérdida 
class Loss(torch.nn.modules.Module):
    def __init__(self, Wt1, Wt0):
        super(Loss, self).__init__()
        self.Wt1 = Wt1
        self.Wt0 = Wt0
        
    def forward(self, inputs, targets, phase):
        loss = - (self.Wt1[phase] * targets * inputs.log() + self.Wt0[phase] * (1 - targets) * (1 - inputs).log())
        return loss

In [25]:
from densenet import densenet169

In [26]:
model = densenet169(pretrained=True)

C:\Users\paula\Documents\Repositorio\Test\densenet.py:115: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [27]:
criterion = Loss(Wt1, Wt0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, verbose=True)


C:\Users\paula\miniforge3\envs\tfg\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [28]:
num_epochs = 5

In [29]:
since = time.time()
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
costs = {x:[] for x in DATA_CAT} # for storing costs per epoch
accs = {x:[] for x in DATA_CAT} # for storing accuracies per epoch
print('Train batches:',  train_num_batches)
print('Valid batches:', valid_num_batches)


Train batches: 1
Valid batches: 1


In [31]:
for epoch in range(num_epochs):
    #confusion_matrix = {x: meter.ConfusionMeter(2, normalized=True) 
    #                    for x in DATA_CAT}
        
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in DATA_CAT:
        model.train(phase=='train')
        running_loss = 0.0
        running_corrects = 0
           
        # Iterate over data.
        for i, data in enumerate(train_dataloader):
            # get the inputs
            print(i, end='\r')
            inputs = data['images'][0]
            labels = data['label'].type(torch.FloatTensor)
            # wrap them in Variable
            inputs = Variable(inputs)
            labels = Variable(labels)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            outputs = model(inputs)
            outputs = torch.mean(outputs)
            loss = criterion(outputs, labels, phase)
            running_loss += loss.data[0]
            # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                optimizer.step()
            # statistics
            preds = (outputs.data > 0.5).type(torch.FloatTensor)
            running_corrects += torch.sum(preds == labels.data)
            #confusion_matrix[phase].add(preds, labels.data)
            
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects / dataset_sizes[phase]
        costs[phase].append(epoch_loss)
        accs[phase].append(epoch_acc)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))
            
            #print('Confusion Meter:\n', confusion_matrix[phase].value())
            
        # deep copy the model
        if phase == 'valid':
            scheduler.step(epoch_loss)
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
    time_elapsed = time.time() - since
    print('Time elapsed: {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
    print()


Epoch 1/5
----------


PermissionError: [Errno 13] Permission denied: 'C:/Users/paula/Documents/Repositorio/MURA_prueba/train/XR_WRIST/patient00021/study1_positive/'